## Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

an MLflow Tracking Server that creates and manages experiments and runs.
an MLflow Registry Server that creates and manages registered models and model versions.
To instantiate it we need to pass a tracking URI and/or a registry URI

In [1]:
from mlflow.tracking import MlflowClient

mlflow_tracking_uri = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=mlflow_tracking_uri)

In [2]:
client.tracking_uri

'sqlite:///mlflow.db'

In [3]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1720948382481, experiment_id='1', last_update_time=1720948382481, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1720948233631, experiment_id='0', last_update_time=1720948233631, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=1,
    filter_string="",
    run_view_type =ViewType.ACTIVE_ONLY ,
    max_results = 5,
    order_by=["metrics.rmse ASC"]
    
)

In [5]:
for run in runs:
    print(f"run id : {run.info.run_id},rmse :{run.data.metrics['rmse']:.4f} ")

run id : 837733a73ab040faaa4ecff356fc5283,rmse :6.3071 
run id : 19dda245c7df4764aad5f2456fd89e9a,rmse :6.7423 
run id : c3464e4091ec4b68a2709351b256d00c,rmse :6.9036 
run id : 0e740465f4a64fdaa97045af2de67b4b,rmse :6.9139 
run id : 9d63f589d1104960a8c918a69d6480ec,rmse :804.5511 


## Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

Register a new version for the experiment nyc-taxi-regressor
Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 3 was created.
Transition the version 3 to "Staging" and adding annotations to it.

In [18]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1720948382481, experiment_id='1', last_update_time=1720948382481, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
run_id = "c3464e4091ec4b68a2709351b256d00c"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name="nyc-taxi-experiment")

Registered model 'nyc-taxi-experiment' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-experiment'.


<ModelVersion: aliases=[], creation_timestamp=1720960873010, current_stage='None', description=None, last_updated_timestamp=1720960873010, name='nyc-taxi-experiment', run_id='c3464e4091ec4b68a2709351b256d00c', run_link=None, source='mlflow-artifacts:/1/c3464e4091ec4b68a2709351b256d00c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [8]:
# Get the list of registered models
models = client.search_registered_models()
for model in models:
    print(f"Model name: {model.name}")
    for version in model.latest_versions:
        print(f" - Version: {version.version}, Status: {version.current_stage}")


Model name: nyc-taxi-experiment
 - Version: 2, Status: Staging
 - Version: 3, Status: None


In [9]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1720960650645, description='', last_updated_timestamp=1720960873010, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1720960659739, current_stage='Staging', description='', last_updated_timestamp=1720960701922, name='nyc-taxi-experiment', run_id='19dda245c7df4764aad5f2456fd89e9a', run_link='', source='mlflow-artifacts:/1/19dda245c7df4764aad5f2456fd89e9a/artifacts/modles_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <ModelVersion: aliases=[], creation_timestamp=1720960873010, current_stage='None', description=None, last_updated_timestamp=1720960873010, name='nyc-taxi-experiment', run_id='c3464e4091ec4b68a2709351b256d00c', run_link=None, source='mlflow-artifacts:/1/c3464e4091ec4b68a2709351b256d00c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-experiment', tags={}>]

In [10]:
# Parameters for the model version to be transitioned
model_name = "nyc-taxi-experiment"
model_version = "3"
new_stage = "Staging"  # Example stages: "None", "Staging", "Production", "Archived"

# Update the model version stage
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

print(f"Model '{model_name}' version '{model_version}' has been transitioned to stage '{new_stage}'.")

Model 'nyc-taxi-experiment' version '3' has been transitioned to stage 'Staging'.


/tmp/ipykernel_93444/804856691.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [11]:
# update the Description of the model

import datetime

date = datetime.date.today()

# Parameters for the model version to be transitioned
model_name = "nyc-taxi-experiment"
model_version = "3"
new_stage = "Staging"  # Example stages: "None", "Staging", "Production", "Archived"

# Update the model version stage
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The Model Version {model_version} was transitioned to {new_stage} on {date}"
)


<ModelVersion: aliases=[], creation_timestamp=1720960873010, current_stage='Staging', description='The Model Version 3 was transitioned to Staging on 2024-07-14', last_updated_timestamp=1720960933906, name='nyc-taxi-experiment', run_id='c3464e4091ec4b68a2709351b256d00c', run_link=None, source='mlflow-artifacts:/1/c3464e4091ec4b68a2709351b256d00c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

## Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

*Note*: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [33]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

#     df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
#     df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, logged_model, X_test, y_test):
    model = mlflow.pyfunc.load_model(logged_model)
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [13]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [20]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/Mlops-zoomcamp/2. Experiment tracking and model management/preprocessor'

In [23]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [24]:
X_test = preprocess(df, dv)

In [25]:
target = "duration"
y_test = df[target].values

In [34]:
logged_model = 'runs:/837733a73ab040faaa4ecff356fc5283/modles_mlflow'

%time test_model(name=model_name, logged_model=logged_model, X_test=X_test, y_test=y_test)

CPU times: user 13 s, sys: 118 ms, total: 13.1 s
Wall time: 7.34 s


/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.257742755585545}

In [35]:
logged_model = 'runs:/c3464e4091ec4b68a2709351b256d00c/modles_mlflow'

%time test_model(name=model_name, logged_model=logged_model, X_test=X_test, y_test=y_test)

CPU times: user 8.56 s, sys: 2.53 s, total: 11.1 s
Wall time: 16.6 s


/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.880226863989569}

In [ ]:
# We can clearly see above still our Production model is better than staging model so we didn't need to migrate to Production.

client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)